# Lab 1: Summarization model Training

## Introduction
---

본 모듈에서는 허깅페이스 트랜스포머(Hugging Face transformers) 라이브러리를 사용하여 문장 요약 모델을 훈련합니다. 
한국어 핸즈온을 위해, 필자가 직접 네이버 뉴스를 크롤링하여 데이터셋을 생성하였으며, 이 데이터셋은 비영리 목적으로 자유롭게 사용하실 수 있습니다.

### References

- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- Summarization fine-tuning: https://huggingface.co/docs/transformers/tasks/summarization


## 1. Setup Environments
---

### Import modules

In [1]:
import os
import sys
import json
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Trainer, TrainingArguments, set_seed
)
from transformers import T5Model, T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers.trainer_utils import get_last_checkpoint
from datasets import load_dataset, load_metric, ClassLabel, Sequence

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Argument parser


In [2]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=8)
    parser.add_argument("--eval_batch_size", type=int, default=32)
    parser.add_argument("--max_length", type=int, default=128)
    parser.add_argument("--stride", type=int, default=32)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=3e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--debug", type=bool, default=False)    
    parser.add_argument("--tokenizer_id", type=str, default='digit82/kolang-t5-base')
    parser.add_argument("--model_id", type=str, default='digit82/kolang-t5-base')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

In [3]:
train_dir = 'seq2seq_summarize_train'
valid_dir = 'seq2seq_summarize_valid'
!rm -rf {train_dir} {valid_dir}
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 `cd scripts & python3 train.py` 커맨드로 훈련 스크립트를 실행할 수 있습니다.

In [4]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'
num_gpus = torch.cuda.device_count()

!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(num_gpus)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'

args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

[{204499775.py:21} INFO - ***** Arguments *****
[{204499775.py:22} INFO - epochs=1
seed=42
train_batch_size=8
eval_batch_size=32
max_length=128
stride=32
warmup_steps=100
logging_steps=100
learning_rate=3e-05
disable_tqdm=False
fp16=True
debug=False
tokenizer_id=digit82/kolang-t5-base
model_id=digit82/kolang-t5-base
output_data_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/summarization/data
model_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/summarization/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/summarization/seq2seq_summarize_train
valid_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/summarization/seq2seq_summarize_valid
chkpt_dir=chkpt



<br>

## 2. Preparation & Custructing Feature set
---

### Dataset

본 핸즈온의 데이터셋은 필자가 직접 만든 데이터로 2022년 7월 1일~7월 10일 네이버 뉴스 중 IT과학 및 경제 뉴스를 크롤링하였습니다.
뉴스 요약 데이터가 별도로 없기에, 정답 데이터는 SKT KoBART 모델 (https://github.com/SKT-AI/KoBART) 로 배치 추론을 수행한 결괏값을 정답 데이터로 간주하였습니다.

In [5]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset('daekeun-ml/naver-news-summarization-ko')
# raw_datasets = load_dataset(
#     "csv",
#     data_files={
#         "train": "raw/naver_news_202207_train.csv",
#         "valid": "raw/naver_news_202207_valid.csv",
#         "test": "raw/naver_news_202207_test.csv",
#     },
# )

[{builder.py:463} WARNING - Using custom data configuration daekeun-ml--naver-news-summarization-ko-d143237f676031cc


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-d143237f676031cc/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [7]:
show_random_elements(raw_datasets["train"], num_examples=1)

,date,category,press,title,document,link,summary
0,2022-07-07 06:01:30,IT과학,조선비즈,車전장에서 맞붙은 LG·삼성 경쟁과 협업으로 판 키운다,LG전자 상반기 수주액 8조원 삼성전자 반도체 중심의 전장 사업 가전 등에서 쌓은 기술력 자동차로 옮겨 메르세데스 벤츠 전동화 브랜드 메르세데스 EQ의 플래그십 EQS에 적용된 LG전자 인포테인먼트 시스템. LG전자 제공 LG와 삼성이 전장 電裝·자동차 전기장치 부품 으로 전장 戰場 을 옮겼다. 가전 스마트폰 등으로 쌓은 전기 전자 기술력을 자율주행 전기차 등 미래 모빌리티에 접목해 새 먹거리로 삼겠다는 것이다. 단순 이동수단이었던 자동차가 움직이는 전자제품으로 변하고 있는 최근의 추세와 무관치 않다. 다만 아직 두 회사는 전장 분야에서 큰 성과를 거두지는 못한 상태다. 자동차 산업은 특성상 진입 문턱이 높아 장기적인 관점에서의 시장 진출이 필요하다는 분석이 나온다. LG전자는 최근 올해 상반기 자동차 전장 신규 수주가 8조원에 달한다고 밝혔다. 오디오·비디오·내비게이션 AVN 이 중심이 되는 인포테인먼트 시스템과 5세대 이동통신 5G 텔레매틱스를 유럽 완성차 업체와 일본 업체에 각각 공급하게 된 덕분이다. 상반기 신규 수주액 8조원은 지난해 LG전자가 달성한 총수주액 60조원의 약 13%에 해당한다. LG전자는 올해 회사 전체 수주 잔고가 65조원에 이를 것으로 전망하고 있다. LG전자는 전장을 전담하는 VS 비히클솔루션 사업본부와 자회사 ZKW의 자동차 조명 세계 3위 차 부품 업체 마그나와 합작해 설립한 LG마그나 이 e 파워트레인의 전기차 동력계를 전장 주축으로 삼고 있다. 삼성전자와 하만이 공동개발한 디지털 콕핏 2021. 삼성전자 제공 삼성전자도 자동차 전장 분야에 대한 꾸준한 투자와 관심을 보내고 있다. 지난 2017년 세계 선두권의 전자업체인 하만을 인수한 시점부터 이 분야 역량을 키워가고 있다. 주력인 반도체 사업에서 자동차 비중을 높이고 있다. 테슬라 자율주행 시스템에 들어가는 칩 등을 공급한다. LG와 삼성은 가전 등에서 경쟁 구도를 형성하고 있고 여기에 쌓은 기술적 노하우를 전장에 활용하고 있다. 그만큼 경쟁 시장에서 맞붙을 여지가 크다. 하지만 경쟁과 함께 일정 부분 협력도 있다. 폭스바겐그룹에 채택된 LG전자의 인포테인먼트 시스템에 삼성전자 LSI사업부가 설계한 ‘엑시노트 오토 V7′을 장착하는 식이다. 모바일 칩 설계 역량 노하우를 갖고 있는 삼성전자가 LG전자 인포테인먼트 시스템의 두뇌를 만든 것이다. 이를 토대로 두 회사가 더 넓은 분야에서의 협력이 가능하다는 얘기가 나온다. LG와 삼성이 자동차 전장 분야에서 기존의 자동차 부품 회사와 다른 점은 그룹 계열사와 관계사 협력과 부품과 기술의 수직계열화가 가능하다는 점이 이점이다. LG전자가 통신 텔레매틱스 과 인포테인먼트 시스템 전기차 동력계 조명을 만들고 여기에 필요한 디스플레이를 LG디스플레이가 LG이노텍이 첨단운전보조시스템 ADAS 용 카메라 조명용 발광다이오드 LED 모터 등을 납품하는 식이다. LG CNS는 네트워크 보안과 전기차 충전 인프라를 구축하고 LG에너지솔루션은 전기차 배터리를 만든다. 또 계열분리된 LX그룹의 LX하우시스는 자동차 내외장재 LX세미콘은 자동차용 마이크로콤포넌트 MC 등을 공급한다. LG마그나 이 e 파워트레인의 전기차 동력게 콘셉트 사진. LG전자 제공 삼성 또한 이런 식으로 자동차 전장 부품 시장에 뛰어들고 있다. 자회사 하만을 중심으로 삼성디스플레이 디스플레이 와 삼성SDS 물류 삼성SDI 배터리 삼성전기 카메라모듈 MLCC 등이 전장 사업을 함께 한다. 이 가운데 삼성SDS의 경우 자동차 전장회사에 대한 인수합병 M A 얘기도 나오는 중이다. 삼성전자 자체적으로도 반도체 분야 역량을 활용해 인포테인먼트용 칩 자율주행차용 연산 칩·이미지센서 등에 뛰어들고 있다. 업계 관계자는 “LG와 삼성 모두 다양한 전장 분야 부품의 일괄 공급이 가능하다는 점에서 두 회사는 기존의 자동차 부품사와는 전혀 다른 경쟁력을 가질 수 있게 되는 것이다”라고 했다. 다만 전장 사업 확대와 달리 뚜렷한 매출 성과는 내지 못하고 있다는 점은 한계로 남는다. 전장 공급 수주를 따내도 실제 납품과 매출로 이어지는 비중이 낮은 것이다. LG전자는 2013년 전장 사업을 시작한 이후 2015년 4분기 흑자 50억원 를 기록했던 것을 제외하면 여전히 적자다. 증권가에선 올해 2분기 400억 500억원의 흑자를 예상하고 있지만 갈 길이 먼 상태다. 반도체 공급난으로 수주와 실제 공급 사이의 이익 발생 시차는 더 넓어지고 있다. 영업이익률 역시 마이너스에 머물러 있다. 삼성전자의 전장 축인 하만의 경우에도 삼성전자에 인수되기 전보다 매출이 적어진 상태다. 인수 이후 영업이익률은 0 2%대를 유지하고 있다. 지난해 말 영업이익률이 5.95%로 올랐으나 매출이 늘었다기보다 원가절감에 따른 것으로 해석된다. 이 가운데 반도체 공급난은 상황을 더욱 악화시키는 요인이 되고 있다. 삼성전자 자동차용 픽셀 PixCell LED. 삼성전자 제공 업계는 LG와 삼성이 각각 사업 분야에서 글로벌 선두에 서 있는 만큼 매출과 이익이 오르는 것은 시간문제로 본다. 이 때문에 보다 장기적인 관점에서 사업을 할 필요가 있다는 해석을 내놓는다. 업계 관계자는 “자동차 산업은 진입장벽이 높다는 산업적 특성으로 비교적 후발주자인 LG와 삼성의 영향력이 단기간 올라오기 힘든 구조다”라며 “그럼에도 두 회사의 기술적 역량은 글로벌 선두권이기 때문에 비교적 빠른 시기에 시장 안착이 가능하리라 보고 장기적인 관점에서 도전을 이어가는 것이 중요하다”고 했다.,https://n.news.naver.com/mnews/article/366/0000825880?sid=105,최근와 자동차 산업에서 경쟁 구도를 형성하고 있는 두 회사는 가전 스마트폰 등으로 쌓은 전기 전자 기술력을 자율주행 전기차 등 미래 모빌리티에 접목해 새 먹거리로 삼기 위해 전장 電裝·자동차 전기장치 부품 으로 전장 戰場 을 옮기는 시도를 하고 있지만 아직 두 회사는 전장 분야에서 큰 성과를 거두지는 못한 상태다.


In [8]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=42)
split_datasets["validation"] = split_datasets.pop("test")

### Tokenization
데이터셋을 토큰화합니다. 원문과 타겟 번역문 모두 토큰화가 필요하며, 타겟 번역문은 context manager 내에서 `as_target_tokenizer()`로 래핑해야 합니다.

토큰화에 대한 자세한 내용은 https://huggingface.co/docs/datasets/process#processing-data-with-map 를 참조하세요.

In [9]:
tokenizer = T5Tokenizer.from_pretrained(args.model_id)

#### Tokenize Sample Data

In [10]:
document = split_datasets["train"][10]["document"]
summary = split_datasets["train"][10]["summary"]
inputs = tokenizer(document)

# Setup the tokenizer for targets;
# If you forget to tokenize the target within the context manager, the target is tokenized by the input tokenizer. 
with tokenizer.as_target_tokenizer():
    targets = tokenizer(summary)
    
inputs, targets   

({'input_ids': [28775, 3847, 33506, 1081, 33542, 2095, 10063, 34285, 33510, 1192, 5358, 3777, 33588, 5295, 5110, 2755, 9673, 33508, 9092, 33557, 1055, 33531, 1278, 25255, 1169, 1390, 23742, 28992, 1014, 2407, 28231, 2802, 9010, 2195, 8811, 33507, 33508, 9347, 33531, 28775, 3847, 3940, 1342, 1016, 2420, 16076, 15027, 4014, 7141, 1905, 5044, 33508, 33597, 33757, 4150, 1048, 33570, 12930, 18052, 1285, 10125, 6947, 33508, 33650, 33757, 4654, 1834, 3994, 3430, 1439, 8834, 33556, 1816, 14097, 2419, 14278, 1115, 1591, 1094, 1301, 1770, 9092, 33558, 26041, 3955, 5251, 33594, 5970, 1695, 1723, 1342, 1016, 2420, 1325, 11821, 1591, 4276, 33521, 2060, 7911, 1457, 2135, 8485, 2436, 1596, 1325, 10535, 1760, 22136, 2127, 5961, 1706, 1091, 1597, 9264, 33515, 2143, 1115, 1845, 1164, 1237, 1447, 1663, 33508, 1156, 1014, 33702, 10482, 33509, 1081, 33883, 3153, 33910, 6127, 4653, 33702, 33538, 3431, 6653, 9654, 1025, 1115, 3139, 1094, 1029, 2163, 33508, 9347, 33531, 3940, 1342, 1989, 2946, 1439, 2430, 339

In [ ]:
max_input_length = 512
max_target_length = 128
prefix = "summarize:"

# tokenizer helper function
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets;
    # If you forget to tokenize the target within the context manager, the target is tokenized by the input tokenizer. 
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function, 
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
# train_dir = 'datasets/train'
# valid_dir = 'datasets/valid'
# !rm -rf {train_dir} {valid_dir}

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(valid_dir, exist_ok=True) 

train_dataset = tokenized_datasets['train']
valid_dataset = tokenized_datasets['validation']

if not os.listdir(args.train_dir):
    train_dataset.save_to_disk(args.train_dir)
if not os.listdir(args.valid_dir):
    valid_dataset.save_to_disk(args.valid_dir)

# from datasets import load_from_disk
# train_dataset = load_from_disk(args.train_dir)
# valid_dataset = load_from_disk(args.valid_dir)

<br>

## 3. Training (Fine-tuning)
---

### Define Custom metric
특정 시점마다(예: epoch, steps) 검증 데이터셋으로 정밀도(precision), 재현율(recall), F1 스코어, 정확도(accuracy)를 등의 지표를 계산하기 위한 커스텀 함수를 정의합니다.

커스텀 함수의 첫번째 인자는 `EvalPrediction` 객체로, 예측값(`predictions`)과 정답값(`label_ids`)를 포함합니다. 자세한 내용은 아래 웹사이트를 참조하세요. 
https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction


### ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metric
ROUGE는 문장 요약에서 많이 사용하는 대표적인 지표로, 정답 문장의 토큰이 생성(레이블) 토큰에 포함되는 정도를 포함한 n-gram 기반 recall 지표입니다.
(BLEU와 유사하지만, BLEU는 생성된 문장의 토큰이 정답 토큰에 포함되는 정도를 정량화화하는 n-gram precision 지표입니다.) 
ROUGE는 문장 간 중복되는 n-gram(unigram, bigram 등)을 비교하는 ROUGE-N, 두 문장이 최장 길이로 겹치는 문장(LCS; LLongest common subsequence)을 
보는 ROUGE-L, skip-gram을 허용하여 특정 윈도우 크기 내의 단어쌍들을 묶어서 매칭 여부를 판별하는 ROUGE-S 등이 있습니다. 자세한 내용은 논문을 참조하세요.

- ROUGE paper: https://aclanthology.org/W04-1013.pdf

In [13]:
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence 
    # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    # decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

### Pre-trained model

이제 훈련에 필요한 피쳐셋이 모두 준비되었으므로, 사전 훈련된 모델을 로드하여 파인튜닝을 수행합니다.
본 핸즈온에서는 T5 기반 모델을 사용하므로 `T5ForConditionalGeneration` 클래스를 사용하지만, `AutoModelForSeq2SeqLM` 클래스를 사용해도 무방합니다.

In [14]:
model = T5ForConditionalGeneration.from_pretrained(args.model_id)

### Data Collation

동적 길이의 입력 데이터를 처리하기 위해 보통 패딩(padding) 기법을 사용하며, 이 때, 허깅페이스에서 지원하는 데이터 콜레이터(Data Collator)를 사용하면 편리합니다. 번역 모델은 Seq2seq 기반으로 `DataCollatorForSeq2Seq`을 사용하면 됩니다.

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### Training Preparation

Seq2Seq 기반 모델 평가 시에는 `predict_with_generate=True`로 설정하는 것을 잊지 마세요.

참조: https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=args.chkpt_dir,
    overwrite_output_dir=True if get_last_checkpoint(args.chkpt_dir) is not None else False,    
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    weight_decay=0.01,
    learning_rate=float(args.learning_rate),     
    save_total_limit=3,
    predict_with_generate=True,
    fp16=args.fp16,
    disable_tqdm=args.disable_tqdm,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

In [17]:
# For debug only
train_dataset = train_dataset.shuffle(seed=42).select(range(3000))
valid_dataset = valid_dataset.shuffle(seed=42).select(range(100))

[{arrow_dataset.py:3399} WARNING - Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-08ad3de8dc1a3b96/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-ede08f22cb3dcd96.arrow
[{arrow_dataset.py:3399} WARNING - Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-08ad3de8dc1a3b96/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-ee3360bfcdcd073e.arrow


In [18]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


### Training
훈련을 수행합니다. 딥러닝 기반 자연어 처리 모델 훈련에는 GPU가 필수이며, 본격적인 훈련을 위해서는 멀티 GPU 및 분산 훈련을 권장합니다. 만약 멀티 GPU가 장착되어 있다면 Trainer에서 총 배치 크기 = 배치 크기 x GPU 개수로 지정한 다음 데이터 병렬화를 자동으로 수행합니다.

In [19]:
%%time
# train model
if get_last_checkpoint(args.chkpt_dir) is not None:
    logger.info("***** Continue Training *****")
    last_checkpoint = get_last_checkpoint(args.chkpt_dir)
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 94
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.423106,25.493800,8.142300,25.221300,25.306600,18.320000


***** Running Evaluation *****
  Num examples = 100
  Batch size = 128
Saving model checkpoint to chkpt/checkpoint-94
Configuration saved in chkpt/checkpoint-94/config.json
Model weights saved in chkpt/checkpoint-94/pytorch_model.bin
tokenizer config file saved in chkpt/checkpoint-94/tokenizer_config.json
Special tokens file saved in chkpt/checkpoint-94/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 2min 28s, sys: 25.9 s, total: 2min 53s
Wall time: 1min 48s


<br>

## 4. Evaluation
---

평가를 수행합니다.

In [20]:
outputs = trainer.predict(valid_dataset)
eval_results = outputs.metrics

# writes eval result to file which can be accessed later in s3 ouput
with open(os.path.join(args.output_data_dir, "eval_results.txt"), "w") as writer:
    print(f"***** Evaluation results at {args.output_data_dir} *****")
    for key, value in sorted(eval_results.items()):
        writer.write(f"{key} = {value}\n")
        logger.info(f"{key} = {value}\n")

***** Running Prediction *****
  Num examples = 100
  Batch size = 128
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Evaluation results at /home/ec2-user/SageMaker/sm-kornlp-usecases/summarization/data *****
[{4005816495.py:9} INFO - test_gen_len = 18.32

[{4005816495.py:9} INFO - test_loss = 1.4231057167053223

[{4005816495.py:9} INFO - test_rouge1 = 25.4938

[{4005816495.py:9} INFO - test_rouge2 = 8.1423

[{4005816495.py:9} INFO - test_rougeL = 25.2213

[{4005816495.py:9} INFO - test_rougeLsum = 25.3066

[{4005816495.py:9} INFO - test_runtime = 8.4909

[{4005816495.py:9} INFO - test_samples_per_second = 11.777

[{4005816495.py:9} INFO - test_steps_per_second = 0.118



<br>

## 5. Prediction
---

여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [21]:
from transformers import pipeline
summarizer = pipeline(
    task="summarization",
    model=model, 
    tokenizer=tokenizer,
    device=0
)

In [22]:
context = r"""summary:아마존웹서비스는 카카오게임즈가 AWS클라우드 역량을 활용해 게임 데이터 분석 솔루션을 실행하고, 대량의 게임 데이터와 설치 건수, 사용자 유지율과 같은 성과 지표를 분석하고 있다고 설명했다. 
현재 카카오게임즈는 폭증하는 데이터를 저장·분석하기 위한 방법으로 클라우드 오브젝트 스토리지 서비스 '아마존 S3(Amazon Simple Storage Service)' 기반 데이터 레이크(Data Lake)를 구축했다. 또 데이터 분석을 용이하게 해주는 대화형 쿼리 서비스 '아마존 아테나(Amazon Athena)'를 도입해 데이터 레이크로부터 게임 데이터를 통합하고, 게임 사용자 행동과 관련된 인사이트를 확보 중이다. 
이를 통해 카카오게임즈는 게임 봇을 탐지하고 제거하는 방식으로 사용자 경험을 제고했다. 또한 관계형 데이터베이스 서비스 '아마존 오로라(Amazon Aurora)'를 활용해 게임 내 구매와 같은 대규모 데이터베이스 거래를 처리하고 있다. 이밖에도 카카오게임즈는 ML 모델 구축, 교육 및 배포를 위한 완전 관리형 서비스 '아마존 세이지메이커(Amazon SageMaker)'를 활용할 예정이다.
"""
summarizer(context)

[{'summary_text': '카카오게임즈는 AWS클라우드 역량을 활용해 게임 데이터 분석 솔루션을 실행하고, 대량의 게임 데이터와 설치 건수, 사용자 유지율과 같은 성과 지표를 분석하고 있다고 설명했다.'}]

In [23]:
context = r"""summary:아마존웹서비스(AWS)가 AWS 리인벤트(AWS re:Invent) 행사를 통해 업계 선도적인 머신러닝(ML) 서비스인 아마존 세이지메이커의 6가지 신규 기능을 발표했다

이를 기반으로 접근성이 우수하고 비용 효율적인 ML 서비스를 제공하겠다는 복안이다.
이번에 새롭게 발표된 세이지메이커의 주요 기능은 정확한 ML 예측을 생성하기 위한 노코드 환경, 고도로 숙련된 주석자를 활용하는 정확한 데이터 레이블링, 전문 영역간 협업을 강화하는 아마존 세이지메이커 스튜디오 범용 노트북 환경, 코드를 효율화하는 머신러닝 학습을 위한 컴파일러, ML 추론에 대한 자동 컴퓨팅 인스턴스 선택, ML 추론을 위한 서버리스 컴퓨팅을 포함한다. 이곳에서 아마존 세이지메이커를 지금 바로 사용할 수 있다. 
사실상 무한하게 가용할 수 있는 컴퓨팅 용량, 클라우드 환경에서의 방대한 데이터 확산, 개발자용 도구의 급속한 발전에 힘입어 ML은 다양한 산업 분야에서 주류로 자리매김했다.
지난 수년간 AWS는 보다 많은 기업 고객이 손쉽게 ML 서비스에 접근할 수 있도록 노력을 기울여 왔는데, 특히 아마존 세이지메이커는 현재 아스트라제네카, 오로라, 캐피털 원, 서너, 디스커버리, 현대그룹, 인튜이트, 톰슨 로이터, 타이슨, 뱅가드를 비롯한 전 세계 수만 기업 고객이 활용 중인, AWS 역사상 가장 빠른 속도로 성장하는 서비스 중 하나이다.
이들 기업 고객은 매달 수천억 개의 예측을 수행하는 수십억 개의 매개변수로 구성된 모델들을 포함하는 모든 규모의 ML 모델 학습을 위해 아마존 세이지메이커를 이용하고 있다.
기업 고객이 아마존 세이지메이커에서 ML 모델 학습 및 추론을 더욱 확장해 나감에 따라, AWS는 서비스 기능 확대를 위한 지속적인 투자를 단행해 지난해에만 60개 이상의 새로운 아마존 세이지메이커 기능을 공개했다.
앞선 진보에 이어, 아번에 발표된 아마존 세이지메이커의 신규 기능 또한 ML을 위한 데이터를 준비 및 수집하고, 모델 학습 속도를 가속화하며, 추론에 필요한 컴퓨팅 유형과 규모를 최적화함으로써, 보다 많은 사용자가 손쉽게 ML을 활용할 수 있도록 확장 지원하는데 방점을 두고 있다.  
"""
summarizer(context)

[{'summary_text': '머신러닝 서비스인 아마존 세이지메이커의 6가지 신규 기능을 발표하여 접근성이 우수하고 비용 효율적인 ML 서비스를 제공하겠다는 복안이다.'}]

In [24]:
# import torch
# inputs = tokenizer.encode(context)
# inputs_= torch.tensor([inputs]).to('cuda:0')

# sample_outputs = model.generate(
#     inputs_, 
#     do_sample=True,
#     max_length=128,
#     top_k=30, # 확률 순위가 top_k위 밖인 토큰은 샘플링에서 제외
#     top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
# )
# tokenizer.decode(sample_outputs.tolist()[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)